In [1]:
import pyscheduling.FS.FmCmax as fm_cmax
import pyscheduling.FS.FmSijkCmax as fmsi_cmax
from pyscheduling.FS.FlowShop import Job, FlowShopSolution

from docplex.cp.model import CpoModel
from docplex.cp.solver.cpo_callback import CpoCallback
from docplex.cp.expression import INTERVAL_MAX

In [5]:
def csp_transform_solution(msol, E_i, instance ):

    sol = FlowShopSolution(instance)
    for k in range(instance.m):
        k_tasks = []
        for i in range(instance.n):
            start = msol[E_i[i][k]][0]
            end = msol[E_i[i][k]][1]
            k_tasks.append(Job(i,start,end))

            k_tasks = sorted(k_tasks, key= lambda x: x[1])
            sol.machines[k].job_schedule = [job[0] for job in k_tasks]
    
    sol.job_schedule = sol.machines[0].job_schedule
    
    return sol

In [6]:
instance = fmsi_cmax.FmSijkCmax_Instance.generate_random(10,3)
E = range(instance.n)
M = range(instance.m)

model = CpoModel("FS_Model")

# Preparing transition matrices
trans_matrix = {}
if hasattr(instance, 'S'):
    for k in range(instance.m):
        k_matrix = [ [0 for _ in range(instance.n + 1)] for _ in range(instance.n + 1) ]
        for i in range(instance.n):
            ele = instance.S[k][i][i]
            k_matrix[i+1][0] = ele
            k_matrix[0][i+1] = ele

            for j in range(instance.n):
                k_matrix[i+1][j+1] = instance.S[k][i][j]

        trans_matrix[k] = model.transition_matrix(k_matrix)
    
    # Create a dummy job for the first task
    first_task = model.interval_var(size=0, optional= False, start = 0, name=f'first_task')

E_i = [[] for i in E]
M_k = [[] for k in M]
types_k = [ list(range(1, instance.n + 1)) for k in M ]
for i in E:
    for k in M:
        start_period = (instance.R[i], INTERVAL_MAX) if hasattr(instance, 'R') else (0, INTERVAL_MAX)
        job_i = model.interval_var( start = start_period,
                                    size = instance.P[i][k], optional= False, name=f'E[{i},{k}]')
        E_i[i].append(job_i)
        M_k[k].append(job_i)

# No overlap inside machines
seq_array = []
for k in M:
    if hasattr(instance, 'S'):
        seq_k = model.sequence_var([first_task] + M_k[k], [0] + types_k[k], name=f"Seq_{k}")
        model.add( model.no_overlap(seq_k, trans_matrix[k]) )
    else:
        seq_k = model.sequence_var(M_k[k], types_k[k], name=f"Seq_{k}")
        model.add( model.no_overlap(seq_k) )
        
    seq_array.append(seq_k)
    
# Same sequence constraint
for k in range(1, instance.m):
    model.add( model.same_sequence(seq_array[k - 1], seq_array[k]) )

# Precedence constraint between machines for each job
for i in E:
    for k in range(1, instance.m):
        model.add( model.end_before_start(E_i[i][k - 1], E_i[i][k]) )

# Add objective
model.add( model.minimize( model.max(model.end_of(job_i) for i in E for job_i in E_i[i]) ) )

In [7]:
msol = model.solve(LogVerbosity="Normal", Workers=1, TimeLimit=30, LogPeriod=1000000,
                   log_output=True, trace_log=True, add_log_to_solution=True, RelativeOptimalityTolerance=0)

 ! --------------------------------------------------- CP Optimizer 20.1.0.0 --
 ! Minimization problem - 34 variables, 25 constraints
 ! TimeLimit            = 30
 ! Workers              = 1
 ! LogPeriod            = 1000000
 ! RelativeOptimalityTolerance = 0
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 148.6 (before), 148.6 (after)
 !  . Memory usage      : 477.1 kB (before), 477.1 kB (after)
 ! Using sequential search.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed            Branch decision
                        0         34                 -
 + New bound is 215
 *          1916       64  0.00s               (gap is 88.78%)
 *          1829      127  0.00s               (gap is 88.24%)
 *          1823      207  0.00s               (gap is 88.21%)
 *          1812      270  0.00s               (gap is 88.13%)
 *          1809      445  0.02s               (gap

In [9]:
sol = csp_transform_solution(msol, E_i, instance )
sol.cmax()

In [10]:
print(sol)

Objective : 1956
Jobs sequence : 9	2	8	5	7	0	1	6	3	4
Machine_ID | Job_schedule (job_id , start_time , completion_time) | Completion_time
(9, 0, 66) : (2, 66, 244) : (8, 244, 423) : (5, 423, 616) : (7, 616, 770) : (0, 770, 920) : (1, 920, 1073) : (6, 1073, 1244) : (3, 1244, 1440) : (4, 1440, 1633) | 1633
(9, 66, 140) : (2, 244, 438) : (8, 438, 610) : (5, 616, 791) : (7, 791, 951) : (0, 951, 1137) : (1, 1137, 1310) : (6, 1310, 1491) : (3, 1491, 1645) : (4, 1645, 1811) | 1811
(9, 140, 215) : (2, 438, 619) : (8, 619, 825) : (5, 825, 1016) : (7, 1016, 1191) : (0, 1191, 1356) : (1, 1356, 1509) : (6, 1509, 1662) : (3, 1662, 1810) : (4, 1811, 1956) | 1956


In [23]:
instance.P

[[6, 2, 2],
 [2, 6, 3],
 [2, 6, 7],
 [3, 3, 3],
 [7, 3, 2],
 [5, 5, 3],
 [2, 4, 6],
 [5, 5, 2],
 [4, 7, 4],
 [3, 3, 3]]

In [26]:
instance.S[2][6][1]

6

In [32]:
machine = 2
list_events = sorted([(i, msol[M_k[machine][i]]) for i in E], key = lambda x: x[1][0])
print("\n".join(map(str,list_events)))

(2, IntervalVarValue(start=8, end=15, size=7))
(4, IntervalVarValue(start=23, end=25, size=2))
(6, IntervalVarValue(start=33, end=39, size=6))
(1, IntervalVarValue(start=45, end=48, size=3))
(0, IntervalVarValue(start=51, end=53, size=2))
(8, IntervalVarValue(start=59, end=63, size=4))
(9, IntervalVarValue(start=68, end=71, size=3))
(5, IntervalVarValue(start=77, end=80, size=3))
(3, IntervalVarValue(start=85, end=88, size=3))
(7, IntervalVarValue(start=95, end=97, size=2))


In [29]:
instance.to_txt("fs_cmax_instance.txt")

In [30]:
sol.to_txt("fs_cmax_solution.txt")